In [2]:
!nvidia-smi

Tue Sep 30 01:34:20 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   43C    P8             11W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [3]:
!nvidia-smi -L || true

import torch, torchvision
print("torch:", torch.__version__)
print("torchvision:", torchvision.__version__)
print("CUDA available:", torch.cuda.is_available())


GPU 0: Tesla T4 (UUID: GPU-3f5ef6a4-1e4f-ecac-a352-ec20ecfd4813)
torch: 2.8.0+cu126
torchvision: 0.23.0+cu126
CUDA available: True


In [4]:
import os, random, math, time
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.backends.cudnn as cudnn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms, models

# 재현성(속도 위해 cudnn.benchmark=True)
SEED = 42
random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)
cudnn.benchmark = True

device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

In [5]:
# CIFAR-10 통계
CIFAR10_MEAN = (0.4914, 0.4822, 0.4465)
CIFAR10_STD  = (0.2023, 0.1994, 0.2010)

# AutoAugment이 없는 환경 대비 try/except
try:
    AA = transforms.AutoAugment(transforms.autoaugment.AutoAugmentPolicy.CIFAR10)
except Exception as e:
    print("AutoAugment 미지원 → 기본 증강 사용")
    AA = transforms.RandomApply([
        transforms.ColorJitter(0.2, 0.2, 0.2, 0.1)
    ], p=0.8)

train_tfms = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    AA,  # 강한 증강(또는 대체)
    transforms.ToTensor(),
    transforms.Normalize(CIFAR10_MEAN, CIFAR10_STD),
])

test_tfms = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(CIFAR10_MEAN, CIFAR10_STD),
])

data_root = "./data"
train_set = datasets.CIFAR10(root=data_root, train=True, download=True, transform=train_tfms)
test_set  = datasets.CIFAR10(root=data_root, train=False, download=True, transform=test_tfms)

BATCH_SIZE = 128
NUM_WORKERS = 2  # Colab에선 2~4 권장

train_loader = DataLoader(train_set, batch_size=BATCH_SIZE, shuffle=True,
                          num_workers=NUM_WORKERS, pin_memory=True)
test_loader  = DataLoader(test_set, batch_size=256, shuffle=False,
                          num_workers=NUM_WORKERS, pin_memory=True)

len(train_set), len(test_set)


100%|██████████| 170M/170M [00:13<00:00, 12.5MB/s]


(50000, 10000)

In [6]:
# ResNet18 기본 구조에서 CIFAR-10(32x32)에 맞게 첫 Conv와 MaxPool 조정
model = models.resnet18(weights=None, num_classes=10)
model.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False)
model.maxpool = nn.Identity()
model = model.to(device)

sum(p.numel() for p in model.parameters()) / 1e6  # 파라미터(M)


11.173962

In [7]:
EPOCHS = 30
BASE_LR = 0.1
WEIGHT_DECAY = 5e-4
MOMENTUM = 0.9
LABEL_SMOOTH = 0.1

criterion = nn.CrossEntropyLoss(label_smoothing=LABEL_SMOOTH)
optimizer = torch.optim.SGD(model.parameters(),
                            lr=BASE_LR, momentum=MOMENTUM,
                            weight_decay=WEIGHT_DECAY, nesterov=True)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=EPOCHS)

scaler = torch.cuda.amp.GradScaler(enabled=(device == "cuda"))


/tmp/ipython-input-931097847.py:13: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = torch.cuda.amp.GradScaler(enabled=(device == "cuda"))


In [8]:
def train_one_epoch(epoch):
    model.train()
    running_loss, correct, total = 0.0, 0, 0

    for images, targets in train_loader:
        images = images.to(device, non_blocking=True)
        targets = targets.to(device, non_blocking=True)

        optimizer.zero_grad(set_to_none=True)
        with torch.cuda.amp.autocast(enabled=(device=="cuda")):
            outputs = model(images)
            loss = criterion(outputs, targets)

        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()

        running_loss += loss.item() * images.size(0)
        _, preds = outputs.max(1)
        correct += preds.eq(targets).sum().item()
        total += targets.size(0)

    train_loss = running_loss / total
    train_acc = correct / total
    print(f"[Epoch {epoch:02d}] Train: loss={train_loss:.4f}, acc={train_acc*100:.2f}%  lr={scheduler.get_last_lr()[0]:.5f}")
    return train_loss, train_acc

@torch.no_grad()
def evaluate():
    model.eval()
    running_loss, correct, total = 0.0, 0, 0
    for images, targets in test_loader:
        images = images.to(device, non_blocking=True)
        targets = targets.to(device, non_blocking=True)
        outputs = model(images)
        loss = criterion(outputs, targets)
        running_loss += loss.item() * images.size(0)
        _, preds = outputs.max(1)
        correct += preds.eq(targets).sum().item()
        total += targets.size(0)
    test_loss = running_loss / total
    test_acc = correct / total
    return test_loss, test_acc


In [9]:
best_acc = 0.0
best_path = "best_resnet18_cifar10.pth"

for epoch in range(1, EPOCHS + 1):
    train_one_epoch(epoch)
    test_loss, test_acc = evaluate()
    print(f"          Val : loss={test_loss:.4f}, acc={test_acc*100:.2f}%")
    if test_acc > best_acc:
        best_acc = test_acc
        torch.save({"model": model.state_dict(),
                    "acc": best_acc, "epoch": epoch}, best_path)
    scheduler.step()

print(f"\nBest Acc: {best_acc*100:.2f}%  (saved to {best_path})")


/tmp/ipython-input-2588914755.py:10: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(enabled=(device=="cuda")):


[Epoch 01] Train: loss=2.4338, acc=13.64%  lr=0.10000
          Val : loss=2.0378, acc=22.41%
[Epoch 02] Train: loss=2.0077, acc=25.30%  lr=0.09973
          Val : loss=1.7838, acc=39.59%
[Epoch 03] Train: loss=1.8388, acc=36.43%  lr=0.09891
          Val : loss=1.9006, acc=39.51%
[Epoch 04] Train: loss=1.6417, acc=47.22%  lr=0.09755
          Val : loss=1.5609, acc=51.65%
[Epoch 05] Train: loss=1.4880, acc=55.28%  lr=0.09568
          Val : loss=1.4451, acc=57.02%
[Epoch 06] Train: loss=1.3610, acc=61.02%  lr=0.09330
          Val : loss=1.2148, acc=68.00%
[Epoch 07] Train: loss=1.2409, acc=66.86%  lr=0.09045
          Val : loss=1.1191, acc=72.41%
[Epoch 08] Train: loss=1.1695, acc=70.23%  lr=0.08716
          Val : loss=1.0320, acc=76.90%
[Epoch 09] Train: loss=1.1150, acc=72.85%  lr=0.08346
          Val : loss=1.0748, acc=75.45%
[Epoch 10] Train: loss=1.0801, acc=74.69%  lr=0.07939
          Val : loss=1.0044, acc=78.48%
[Epoch 11] Train: loss=1.0482, acc=76.09%  lr=0.07500
      